# Hands on Radar Data, Part 1
### Alyssa Matthews, Marqi Rocque, and Ya-Chien Feng

## Please go to the ARM Jupyterhub page:
## jupyterhub.arm.gov

## Log in
## Choose the ARM Short course 2024 option

## Setup

Welcome to the hands on radar coding portion of the course! 
During or after the course, should you wish to access the notebooks again, you can find them here:

https://github.com/aamatthews/2024_Atmos_Inst



Before we get started, there is a little setup we have to do first. 

In each notebook we will be using PyART, the Python ARM Radar Toolkit, which is a python package that is helpful for reading in, plotting, and contains multiple algorithms for digging in deeper to radar data.

If you are new to Python and Jupyter Notebooks, the code can be split into chunks (cells) and each cell can be run separately. I find this especially useful for creating plots or to create the initial python code before transferring it to one big script.

*   To run a cell, press shift+enter on your keyboard, or click the play button at the top menu of the screen

We will start with learning to read in and plot the various types of ARM radar data - Vertically Pointing, RHIs, and PPIs.

Let's grab some data for today! 
To do this, we will use ACT, a python package in ARM that allows you to grab and work with data from the Data Discovery right in the notebook, no ordering necessary. I won't go into detail about this package here, as you will learn about it more tomorrow, but for now we will just use it as a way to access the data.

#### IMPORTANT!
You will also need an account and token to download data using the ARM Live webservice. Navigate to the [webservice information page](https://adc.arm.gov/armlive/) and log in using your ARM login info to get your token. Your account username will be your ARM username.

In [ ]:
import act

In [ ]:
# Set your username and token here!
username = 'amatthews'
token = '3b66c95f8109abb9'

# Set the datastream and start/enddates
datastream = 'houkazrcfrgeqcM1.b1'
startdate = '2022-07-01T19:00:00'
enddate = '2022-07-01T20:00:00'

# Use ACT to easily download the data.  Watch for the data citation!  Show some support
# for ARM's instrument experts and cite their data if you use it in a publication
result = act.discovery.download_arm_data(username, token, datastream, startdate, enddate)

Next, lets import the libraries we will need in this notebook. We will add more later in the course in future notebooks, but these are the basics I tend to use most often in my work.

In [ ]:
import numpy as np
import netCDF4 as nc
import xarray as xr
import pandas as pd
import pyart

import matplotlib.pyplot as plt
%matplotlib inline

# Reading in files

In this section, I will show you three different ways you can read in files. Which you use really depends on which you prefer or which will work best for your needs.

### Option 1: NetCDF4 library 

For more information, you can view the documentation here:
https://unidata.github.io/netcdf4-python/#Dataset.__init__

In [ ]:
data = nc.Dataset('houkazrcfrgeqcM1.b1/houkazrcfrgeqcM1.b1.20220701.190000.nc')

In the radar files, time is split into base time (the time at the start of the file) and time offset (the time since the file started). I like to save a combined timestamp of the two after reading in the data, as it can make the plots and data easier to interpret later on.

In [ ]:
time =  pd.to_datetime(data.variables['base_time'][:] + data.variables['time_offset'][:], unit='s')

In [ ]:
time[0:10]

Once you have read in the data file, there are various levels you can look at. First, is the data class. This will include all the attributes (or metadata), as well as the variables stored in the file.

In [ ]:
data

Often, it is useful just to see what the variable names are. You can see the full details of each variable with data.variables, but to see just a list of the variable names, you can add the keys() function.

In [ ]:
list(data.variables.keys())

Once you know the name of a variable you want to look at, you can examine it to see the details of that variable, including a more detailed name, the dimension, and units. Lets take a look at Reflectivity.

In [ ]:
data.variables['reflectivity']

Finally, to view the data within the variable, you can add [:]

In [ ]:
data.variables['reflectivity'][:]

Lets view a piece of the attributes or metadata. These are called directly from the dataset.

In [ ]:
data.antenna_altitude

### Option 2: Xarray library 

This is another library that can read in netCDF4 files. Data read in with this library is in a more pandas-like structure (for those familiar with pandas). You can view the documentation for reading in a file with xarray here: 

https://docs.xarray.dev/en/stable/generated/xarray.open_dataset.html#xarray.open_dataset

An ability I particularly like within xarray is its ability to read in multiple data files at once and save them to one structure instead of having to read in each file individually. For this course, though, we will just be focusing on reading in one file at a time for simplicity. The documentation for this can be found at:

https://docs.xarray.dev/en/stable/generated/xarray.open_mfdataset.html#xarray.open_mfdataset

In [ ]:
radar = xr.open_dataset('houkazrcfrgeqcM1.b1/houkazrcfrgeqcM1.b1.20220701.190000.nc', decode_times = False)

One known issue in Xarray is how it handles the 'time' variable in the ARM radar files. To correct this, I overwrite that variable with the combined base_time + time_offset, as I showed previously in the netCDF4 example.

In [ ]:
radar['time'] = pd.to_datetime(radar['base_time'].values + radar['time_offset'].values, unit='s')

To start digging into the data, lets first print the Dataset. As you can see, the output is much more condensed, with many options to expand the data to see what you are interested in. 

In [ ]:
radar

Here, you can click the arrows to expand each header, as well as the page or stack icon to get more info about the variables and data.

Lets next get a list of the available variable names.

In [ ]:
list(radar.variables.keys())

To see the details of a particular variable, you can either expand it out in the radar output, or can use a command identical to the one we used in the netCDF4 library example. Lets look at Reflectivity again, and at the data within that variable.

In [ ]:

radar.variables['reflectivity']

To view the data within the variable, you can add .data or .values to the end of the variable.

In [ ]:
radar.variables['reflectivity'].data

Viewing an attribute is done similarly to what we learned in the netCDF4 library example.

In [ ]:
radar.antenna_diameter

### Option 3: PyART

The PyART package can read in many different types of radar files. You can view its documentation at: 

https://arm-doe.github.io/pyart/

In [ ]:
radar = pyart.io.read('houkazrcfrgeqcM1.b1/houkazrcfrgeqcM1.b1.20220701.190000.nc')

Lets view the information about this radar object. This will show us the variables and their information at the top, and the attributes and metadata at the bottom.

In [ ]:
radar.info()

One difference between PyART and the netCDF4 or Xarray libraries is that PyART calls the data names 'fields' instead of 'variables'. Lets view them now.

In [ ]:
list(radar.fields.keys())

In [ ]:
radar.fields['reflectivity']

To view the data within the variable, you just add ['data'] to the end of the field.

In [ ]:
radar.fields['reflectivity']['data']

Viewing the attributes or metadata is done a little differently to the other two examples when using PyART

In [ ]:
radar.metadata['antenna_altitude']

## Exercise 1
Order up to an hour from TRACER using ACT for one of the following datastreams (you can order more, but it takes time and will take up space in your directory):
houxsacrcfrqcM1.b1 (XSACR)
houkasacrcfrqcM1.b1 (KASACR)
houcsapr2cfrqcS2.b1 (CSAPR2)

Pick one of the methods I showed above and read in a file you just ordered.

Can you find the scan type for that file in the data?

### Hint

it is an attribute or metadata, and not a variable

:)

### Solutions
* Please run the command for any of the datastreams you did not already grab data for below, as we will use it in later notebooks! (rerunning one you already did will not cause any issues or create duplicate files)

In [ ]:
datastream = 'houkasacrcfrqcM1.b1'
startdate = '2022-07-01T19:00:00'
enddate = '2022-07-01T20:00:00'

result = act.discovery.download_arm_data(username, token, datastream, startdate, enddate)

In [ ]:
datastream = 'houxsacrcfrqcM1.b1'
startdate = '2022-07-01T19:00:00'
enddate = '2022-07-01T20:00:00'

result = act.discovery.download_arm_data(username, token, datastream, startdate, enddate)

In [ ]:
datastream = 'houcsapr2cfrqcS2.b1'
startdate = '2022-07-01T19:00:00'
enddate = '2022-07-01T20:00:00'

result = act.discovery.download_arm_data(username, token, datastream, startdate, enddate)

In [ ]:
data = xr.open_dataset('houcsapr2cfrqcS2.b1/houcsapr2cfrqcS2.b1.20220701.192028.nc')

In [ ]:
data.scan_name

In [ ]:
data = nc.Dataset('houcsapr2cfrqcS2.b1/houcsapr2cfrqcS2.b1.20220701.192028.nc')

In [ ]:
data.scan_name

In [ ]:
data = pyart.io.read('houcsapr2cfrqcS2.b1/houcsapr2cfrqcS2.b1.20220701.192028.nc')

In [ ]:
data.metadata['scan_name']

# Now, lets plot some data!


The ARM scanning radars have the ability to scan in multiple ways. Here, we will go over various methods for plotting the most common scan types. First I will show a method with no specialized libraries, then I will show a method using PyART.

![radar scan image](https://journals.ametsoc.org/view/journals/atot/31/3/images/full-jtech-d-13-00044_1-f3.jpg)

## Plot Scan Strategy
First, lets plot a way to view the scan elevations to see what the radar was doing for each PPI or RHI set

In [ ]:
import wradlib as wrl
import os

In [ ]:
data = xr.open_dataset('houcsapr2cfrqcS2.b1/houcsapr2cfrqcS2.b1.20220701.192603.nc')

In [ ]:
data.scan_name

In [ ]:
ranges = data.range.data
elevs = [0.5, 1.5, 3]
site = (-95.284, 29.52, 12)
beamwidth = 0.9

In [ ]:
data.fixed_angle.data

In [ ]:
#Update beam width to match radar
ax = wrl.vis.plot_scan_strategy(ranges, elevs, site, units='km', 
                                maxalt=8000.0, beamwidth=1.25, 
                                )

### Set up environment
#### To get earthdata bearer token: 
- Create free account here: https://urs.earthdata.nasa.gov/home
- click Generate Token
- copy token to the line below, inside the quotes

In [ ]:
os.environ["WRADLIB_EARTHDATA_BEARER_TOKEN"] = ""

Create a folder called wradlib_data in this directory. Wradlib will grab some terrain data from the web and store it here.

In [ ]:
os.environ["WRADLIB_DATA"] = "/data/home/amatthews/Teaching/2024_inst_course/wradlib_data"

In [ ]:
has_data = os.environ.get("WRADLIB_EARTHDATA_BEARER_TOKEN", False)

In [ ]:
if has_data:
    #Create plot of elevations of PPI. 
    #Update azimuth to be the azimuth you want (ie, over KAZR, over a particular mountain, etc)
    ax = wrl.vis.plot_scan_strategy(
        ranges, elevs, site, units="km", terrain=True, maxalt = 8000, az=328
    )


## Vertically pointing data (VPT)

### Method 1

In [ ]:
kazr = xr.open_dataset('houkazrcfrgeqcM1.b1/houkazrcfrgeqcM1.b1.20220701.190000.nc', decode_times = False)
# kazr['time'] = pd.to_datetime(kazr['base_time'].values + kazr['time_offset'].values, unit='s')

In [ ]:
plt.figure(figsize=(15,5))
plt.pcolormesh(kazr['time'], kazr['range'], kazr['reflectivity'].T, cmap = 'jet', vmin=-20, vmax=40)
plt.colorbar(label = 'Reflectivity (dBZ)')
plt.xlabel('Time')
plt.ylabel('Height (m)')
plt.title('KAZR Reflectivity');

### Method 2: PyART

In [ ]:
radar = pyart.io.read('houkazrcfrgeqcM1.b1/houkazrcfrgeqcM1.b1.20220701.190000.nc')

In [ ]:

display = pyart.graph.RadarDisplay(radar)
fig = plt.figure(figsize=[8,4])
display.plot_vpt('reflectivity', vmin=-20, vmax=40)


## RHI or HSRHI

### Method 1

In [ ]:
csapr = xr.open_dataset('houcsapr2cfrqcS2.b1/houcsapr2cfrqcS2.b1.20220701.192028.nc', decode_times = False)
# csapr['time'] = pd.to_datetime(csapr['base_time'].values + csapr['time_offset'].values, unit='s')

In [ ]:
csapr.scan_name

In [ ]:
sweep_start = list(map(int,(csapr['sweep_start_ray_index'].data)))
sweep_end = list(map(int,(csapr['sweep_end_ray_index'].data)))
azi = csapr['azimuth'].data
elv = csapr['elevation'].data
r = csapr['range'].data

y = np.outer(r, np.sin((elv)*np.pi/180.0))
x = np.outer(r, np.cos((elv)*np.pi/180.0))
sweep_num = 0



for i in range(len(sweep_start)):
    sweep_num = i
#     print(sweep_num)
    plt.figure(figsize=(20,40))
    plt.subplot(6,1,1)
    plt.pcolormesh(x[:,sweep_start[sweep_num]:sweep_end[sweep_num]].T,
                   y[:,sweep_start[sweep_num]:sweep_end[sweep_num]].T,
                   csapr['reflectivity'].data[sweep_start[sweep_num]:sweep_end[sweep_num],:], 
                   vmin=-60, vmax=50, cmap='jet')
    clb = plt.colorbar()
    clb.set_label('Zh')
    plt.xlabel('Horizontal Distance from Radar (m)')
    plt.ylabel('Height (m)')
    azi_start = np.nanmedian(azi[sweep_start[sweep_num]])
    plt.title('Azimuth: %f ' % azi_start)
    plt.ylim(0,20000)


### Method 2: PyART

In [ ]:
radar = pyart.io.read('houcsapr2cfrqcS2.b1/houcsapr2cfrqcS2.b1.20220701.192028.nc')

In [ ]:
display = pyart.graph.RadarDisplay(radar)
fig = plt.figure(figsize=[10, 5])

display.plot('reflectivity', 0, vmin=-60, vmax=50.0) #Format is (field, sweep number, minimum value of colorbar, maximum value of colorbar)
# plt.ylim(0,20)

## PPI





### Method 1

This is very similar to the RHI, except instead of y using azimuth, you use elevation and flip from cos to sin (or vice versa)

In [ ]:
ppi = xr.open_dataset('houcsapr2cfrqcS2.b1/houcsapr2cfrqcS2.b1.20220701.192603.nc', decode_times = False)

In [ ]:
ppi.scan_name

In [ ]:
sweep_start = list(map(int,(ppi['sweep_start_ray_index'].data)))
sweep_end = list(map(int,(ppi['sweep_end_ray_index'].data)))
azi = ppi['azimuth'].data
elv = ppi['elevation'].data
r = ppi['range'].data

y = np.outer(r, np.cos((azi)*np.pi/180.0))
x = np.outer(r, np.sin((azi)*np.pi/180.0))
sweep_num = 0

for i in range(len(sweep_start)):
    sweep_num = i
#     print(sweep_num)
    plt.figure(figsize=(10,7.5))

    plt.pcolormesh(x[:,sweep_start[sweep_num]:sweep_end[sweep_num]].T,
                   y[:,sweep_start[sweep_num]:sweep_end[sweep_num]].T,
                   ppi['reflectivity'].data[sweep_start[sweep_num]:sweep_end[sweep_num],:], 
                   vmin=-60, vmax=50, cmap='jet')
    clb = plt.colorbar()
    clb.set_label('Zh')
    plt.xlabel('E-W Distance from Radar (m)')
    plt.ylabel('N-S Distance from Radar (m)')
    elv_start = elv[sweep_start[sweep_num]+50]
    plt.title('Elevation: %f' % elv_start)

### Method 2: PyART

In [ ]:
radar = pyart.io.read('houcsapr2cfrqcS2.b1/houcsapr2cfrqcS2.b1.20220701.192603.nc')

In [ ]:
display = pyart.graph.RadarDisplay(radar)
fig = plt.figure(figsize=[7,6])

display.plot('reflectivity', 2, vmin=-60, vmax=40.)
display.plot_range_rings([20, 40, 60, 80, 100])
display.plot_cross_hair(2.)

## Exercise
1. Plot an XSACR or KASACR scanning file. 

2. What do you see in the data?